In [2]:
from redis import Redis
redis_connection = Redis('this_redis')

# Redis

One interesting use case is to have Redis track the evolution of some long running iterative process.

In [48]:
redis_connection.get('count')

b'6'

In [49]:
redis_connection.get('count')

b'6'

In [50]:
redis_connection.get('count')

b'9'

In [6]:
redis_connection.get('count')

b'19'

In [8]:
redis_connection.get('count')

b'30'

## Passing Parameters Between Processes

We could also use Redis to pass parameters between procesess.

In [26]:
model_params_from_redis = redis_connection.get('model_params')
model_params_from_redis

b'{"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], "penalty": "l1", "solver": "newton-cg"}'

In [14]:
type(model_params_from_redis)

bytes

In [18]:
type(model_params_from_redis.decode())

str

In [19]:
model_params_from_redis.decode()

'{"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], "penalty": "l1", "solver": "newton-cg"}'

In [20]:
import json

In [53]:
json.loads(model_params_from_redis.decode())

{'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
 'penalty': 'l1',
 'solver': 'newton-cg'}

In [54]:
json.loads(model_params_from_redis)

{'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
 'penalty': 'l1',
 'solver': 'newton-cg'}

In [55]:
json.loads(model_params_from_redis.decode()) == json.loads(model_params_from_redis)

True

## Passing Arrays Between Processes

We could also use Redis to pass arrays between procesess.

In [27]:
import numpy as np

In [29]:
A_bytestring = redis_connection.get('encoded_A')
A_bytestring

b'\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00'

In [34]:
A_raveled = np.fromstring(A_bytestring, dtype=int)
A_raveled

array([1, 1, 1, 2, 2, 2, 3, 3, 3])

In [60]:
redis_connection.get('A_n')

b'3'

In [84]:
#Alternative
json.loads(redis_connection.get('A_n'))

3

In [70]:
redis_connection.get('A_n').decode()

'3'

In [39]:
n = int(redis_connection.get('A_n').decode())
m = int(redis_connection.get('A_m').decode())
n, m

(3, 3)

In [40]:
A = A_raveled.reshape(n, m)

In [41]:
A

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [42]:
def matrix_from_redis(redis_client, key, dtype=int):
    matrix_bytestring = redis_client.get(key)
    matrix_raveled = np.fromstring(matrix_bytestring, dtype=dtype)
    n = int(redis_client.get(key+'_n').decode())
    m = int(redis_client.get(key+'_m').decode())
    matrix = matrix_raveled.reshape(n, m)
    return matrix

In [43]:
matrix_from_redis(redis_connection, 'new_A')

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [80]:
redis_connection.get('B')

b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\x03K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i8q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89CH\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.'

This could also be done via the `pickle` library.

In [82]:
import pickle

B_pkl = redis_connection.get('B')
pickle.loads(B_pkl)

array([[1, 3, 4],
       [3, 8, 2],
       [7, 4, 8]])